## Project Background

The NYC Budget Expense Analysis and Optimization project was undertaken to address the challenges associated with managing the city's extensive and complex financial operations. The project aimed to leverage advanced data analytics to uncover opportunities for cost savings and efficiency improvements within New York City's government agencies. The analysis focused on comparing adopted and financial plan expense data to assess budget execution efficiency, identify areas of overspending, and propose rational budget adjustments. Utilizing a publicly available dataset from NYC Open Data, which is updated regularly, the project provided valuable insights for city planners, policymakers, and civic tech organizations to optimize resource allocation and enhance budget management strategies.

## Technology Adoption

The choice of PostgreSQL and MongoDB for this project was driven by their complementary strengths. PostgreSQL, a robust SQL database, was selected for its ability to handle structured data with high accuracy and efficiency, ensuring reliable data integrity and performance. It was ideal for managing and querying the normalized tables that formed the backbone of the expense analysis. On the other hand, MongoDB, a NoSQL database, was chosen for its flexibility in handling unstructured and semi-structured data. Its scalability and ability to efficiently store and process large volumes of diverse data made it a perfect fit for the project's needs, especially in managing the varied datasets involved in the budget analysis. Together, these technologies provided a balanced and scalable solution, enabling the project to deliver detailed and accurate insights into NYC's budget expenses.

The original data run by the NYC government can be found: https://data.cityofnewyork.us/City-Government/Expense-Budget/mwzb-yiwb/about_data

*Disclaimer*: This is a student project. Some of the code in the visualization are courtesy of my teammate Leyi Shi. The MangoDB and Postgre have been set up by Jianfei Shi in the preliminary set up stage. ETL pipeline and data cleaning was primarily done by Yumeng Tian. This Project was conducted in August 2023, and the data were up to Aug 2023.

In [13]:
#importing the neccessary packages
import numpy as np
import pandas as pd
import random
import requests
import json
import plotly.graph_objects as go
import plotly.offline as plotly
import findspark
import psycopg2
import os
import matplotlib.pyplot as plt
import mpld3
import pymongo
from pymongo import MongoClient
from sodapy import Socrata
from pyspark.sql import SparkSession
from flask import Flask, request, render_template, jsonify
from sqlalchemy import create_engine
import plotly.graph_objects as go
import plotly.offline as plotly
import plotly.express as px
import re

print("import complete")

import complete


### Connecting to Database
In this project, my teammate has stored data in PostgreSQL and MongoDB, which allowed us to efficiently manage and query both structured and unstructured data. By connecting to these databases, we were able to leverage the strengths of each system—using PostgreSQL for precise, structured data analysis and MongoDB for handling diverse, semi-structured datasets—thereby enabling a comprehensive exploration of NYC's budget expenses.


In [14]:
nyc_expense_df = pd.read_csv('Expense_Budget.csv')
print("complete")
#MongoDB 
client = MongoClient("localhost", 27017)
db = client.ManagingData_database
collection = db.expense_collection
#Postgre
conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="my_db",
    user="postgres",
    password="123")

/var/folders/34/43gk_4tn6jq6397njd4_cm8h0000gn/T/ipykernel_19171/4114311305.py:1: DtypeWarning:

Columns (2,13) have mixed types. Specify dtype option on import or set low_memory=False.



complete


### FLASK SQL Query for Retrieving Data

The code snippet provided is a Flask SQL query implementation used to retrieve and filter records. The function dynamically builds an SQL query based on user-defined filters, such as *fiscal year*, *agency name*, *unit appropriation name*, *budget code name*, *object class name*, *object code name*, and *responsibility center name*. This allows for flexible and targeted data retrieval, the results was limited to 500 records for efficiency.


In [15]:
### FLASK SQL QUERY for retrieving data
import psycopg2

def get_filtered_records(filters):
    query = "SELECT * FROM new_expense_budget WHERE TRUE"
    filter_clauses = []

    if filters.get('fiscal_year'):
        filter_clauses.append(f"fiscal_year = '{filters['fiscal_year']}'")
    if filters.get('agency'):
        filter_clauses.append(f"agency_name = '{filters['agency']}'")
    if filters.get('unit_appropriation_name'):
        filter_clauses.append(f"unit_appropriation_name = '{filters['unit_appropriation_name']}'")
    if filters.get('budget_code_name'):
        filter_clauses.append(f"budget_code_name = '{filters['budget_code_name']}'")
    if filters.get('object_class_name'):
        filter_clauses.append(f"object_class_name = '{filters['object_class_name']}'")
    if filters.get('object_code_name'):
        filter_clauses.append(f"object_code_name = '{filters['object_code_name']}'")
    if filters.get('responsibility_center_name'):
        filter_clauses.append(f"responsibility_center_name = '{filters['responsibility_center_name']}'")

    if filter_clauses:
        query += " AND " + " AND ".join(filter_clauses)

    # Limit records to 500
    query += " LIMIT 500"

    # Retrieve expense records from PostgreSQL using the built SQL query
    cur = conn.cursor()
    cur.execute(query)
    expense_records = cur.fetchall()
    cur.close()

    return expense_records


#populating the filters dropdown
def get_filter_options():
    cur = conn.cursor()
    cur.execute("SELECT DISTINCT fiscal_year FROM new_expense_budget ORDER BY fiscal_year DESC")
    fiscal_years = cur.fetchall()

    cur.execute("SELECT DISTINCT agency_name FROM agency ORDER BY agency_name ASC")
    agencies = cur.fetchall()

    cur.execute("SELECT DISTINCT unit_appropriation_name FROM unit_appropriation ORDER BY unit_appropriation_name ASC")
    unit_appropriation_names = cur.fetchall()

    cur.execute("SELECT DISTINCT budget_code_name FROM budget_code ORDER BY budget_code_name")
    budget_code_names = cur.fetchall()

    cur.execute("SELECT DISTINCT object_class_name FROM object_class ORDER BY object_class_name")
    object_class_names = cur.fetchall()

    cur.execute("SELECT DISTINCT object_code_name FROM object_code ORDER BY object_code_name")
    object_code_names = cur.fetchall()

    cur.execute("SELECT DISTINCT responsibility_center_name FROM new_expense_budget ORDER BY responsibility_center_name")
    responsibility_center_names = cur.fetchall()

    cur.close()

    return fiscal_years, agencies, unit_appropriation_names, budget_code_names, object_class_names, object_code_names, responsibility_center_names


### Visualizations
In our Flask web application, we integrated five key visualizations to enhance the analysis and user experience, providing users with an intuitive way to explore NYC budget expense data:

1. **Top 10 Agencies by Budget (by Fiscal Year)**: This visualization retrieves data from MongoDB to display the top 10 agencies with the highest budgets for a selected fiscal year. Using Plotly, we generate an interactive bar chart that highlights the distribution of budget allocations among these leading agencies.

2. **Agency Budget Trend (by Fiscal Year)**: This visualization allows users to track the adopted budget trends for a specific agency across multiple fiscal years. The bar chart, generated by Plotly, provides insights into how an agency's budget has evolved over time.

3. **Top 10 Agencies by Total Budget Expense**: This bar chart ranks the top 10 agencies by their aggregated budget expenses. By querying MongoDB, we extract and visualize the data to offer a clear view of which agencies dominate in terms of overall budget expenditure.

4. **Highest Expense-to-Planning Ratio**: This visualization identifies the top 10 agencies with the highest ratios of actual budget expenses to financial planning amounts. The bar chart generated by Plotly allows users to quickly assess which agencies are exceeding their planned budgets.

5. **Lowest Expense-to-Planning Ratio**: Conversely, this bar chart highlights the top 10 agencies with the lowest expense-to-planning ratios. This visualization is crucial for identifying agencies that are spending well below their planned budgets, indicating potential underutilization of allocated resources.

The interactive nature of Plotly's charts allows users to dynamically explore the data, adjust filters, and gain actionable insights with ease. This combination of advanced data visualization and user-centric design makes the application a powerful tool for budget analysis and decision-making.

In [16]:
### FLASK plot_analysis Top 10 Agencies (by fiscal year) 
from pymongo import MongoClient
import plotly.express as px

def top_agencies(fiscal_year):
    client = MongoClient("localhost", 27017)
    db = client.ManagingData_database
    collection = db.expense_collection
    # Query the database to get the top 10 agencies based on the fiscal year
    pipeline = [
        {"$match": {"fiscal_year": fiscal_year}
        },{"$group": {"_id": "$agency_name","total_budget": { "$sum": { "$convert": { "input": "$adopted_budget_amount", "to": "int", "onError": 0 } } }
            }},
        {"$sort": {"total_budget": -1}
        },{"$limit": 10}
    ]
    top_10_agencies = list(collection.aggregate(pipeline))
    # Extract agency names and total budgets for the Plotly graph
    agency_names = [agency["_id"] for agency in top_10_agencies]
    total_budgets = [agency["total_budget"] for agency in top_10_agencies]

    # Create the Plotly graph
    fig = px.bar(x=agency_names, y=total_budgets, labels={"x": "Agency Name", "y": "Total Budget"},
                 title=f"Top 10 Agencies in Fiscal Year {fiscal_year}")

    graph_html = plot(fig, output_type='div')

    return graph_html


#top_agencies("2023")

In [17]:
###FLASK plot_analysis agency trend 
from pymongo import MongoClient
import plotly.express as px
import re
from plotly.offline import plot

def get_agency_budget_by_fiscal_year(agency_name):
    escaped_agency_name = re.escape(agency_name)
    client = MongoClient("localhost", 27017)
    db = client.ManagingData_database
    collection = db.expense_collection
    pipeline = [
        {"$match": {"agency_name": agency_name}},
        {"$group": {"_id": "$fiscal_year",
                "adopted_budget_amount": { "$sum": { "$toDouble": { "$ifNull": ["$adopted_budget_amount", 0] } } }}
        },{"$sort": {"_id": -1}}
    ]

    agency_budget_by_fiscal_year = list(collection.aggregate(pipeline))
    fiscal_years = [data["_id"] for data in agency_budget_by_fiscal_year]
    adopted_budget_amounts = [data["adopted_budget_amount"] for data in agency_budget_by_fiscal_year]

    # Create the Plotly graph
    fig = px.bar(x=fiscal_years, y=adopted_budget_amounts, labels={"x": "Fiscal Year", "y": "Adopted Budget Amount"},
                 title=f"{agency_name} Adopted Budget Amount Trend (by fiscal year)")
    graph_html = plot(fig, output_type='div')
    return graph_html

In [18]:
### 
def get_top_10_agencies():
    # MongoDB query to get top 10 agencies by expense
    client = MongoClient("localhost", 27017)
    db = client.ManagingData_database
    collection = db.expense_collection
    pipeline = [
        {"$group": {"_id": "$agency_name","total_budget": {"$sum": {"$toDouble": "$adopted_budget_amount"}}}
            },{"$sort": {"total_budget": -1}  # Sort desc
            },{"$limit": 10}
    ]

    top_10_agencies = list(collection.aggregate(pipeline))
    #create plotly 
    fig = go.Figure([go.Bar(x=[agency['_id'] for agency in top_10_agencies], y=[agency['total_budget'] for agency in top_10_agencies])])
    fig.update_layout(
        title='Top 10 Agencies by Budget Expense',
        xaxis_title='Agency Name',
        yaxis_title='Aggregated Budget Expense'
    )
    y_range_padding = 0.1
    fig.update_yaxes(range=[0 - y_range_padding, max([agency['total_budget'] for agency in top_10_agencies]) * (1 + y_range_padding)])

    return fig

In [19]:
### highest_ratio_plot
### Courtesy of Leyi
import plotly.graph_objects as go
import plotly.offline as plotly
def higest_ratio():
    client = MongoClient("localhost", 27017)
    db = client.ManagingData_database
    collection = db.expense_collection
    pipeline = [
        {
            '$group': {
                '_id': '$agency_name',
                'total_budget_expense': {'$sum': {'$toDouble': '$adopted_budget_amount'}},
                'total_financial_planning': {'$sum': {'$toDouble': '$financial_plan_amount'}}
            }
        },
        {
            '$match': {
                'total_financial_planning': {'$ne': 0}  
            }
        },
        {
            '$sort': {
                'total_budget_expense': -1 
            }
        },
        {
            '$limit': 10
        },
        {
            '$project': {
                'agency_name': '$_id',
                'budget_expense': '$total_budget_expense',
                'financial_planning': '$total_financial_planning',
                'expense_to_planning_ratio': {'$divide': ['$total_budget_expense', '$total_financial_planning']}
            }
        }
    ]

    result = collection.aggregate(pipeline)
    agency_names = []
    expense_to_planning_ratios = []
    for doc in result:
        agency_names.append(doc['agency_name'])
        expense_to_planning_ratios.append(doc['expense_to_planning_ratio'])

    if not expense_to_planning_ratios or all(ratio == 0 for ratio in expense_to_planning_ratios):
        print("No non-zero expense-to-planning ratio data to plot.")
    else:
        fig = go.Figure([go.Bar(x=agency_names, y=expense_to_planning_ratios)])

        fig.update_layout(
            title='Top 10 Agencies by Highest Expense-to-Planning Ratio',
            xaxis_title='Agency Name',
            yaxis_title='Expense-to-Planning Ratio',
            hovermode='x', 
        )

        y_range_padding = 0.1
        fig.update_yaxes(range=[0, max(expense_to_planning_ratios, default=1) * (1 + y_range_padding)])

        return fig


In [20]:
### FLASK Lowest Ratio Part
### Courtesy of Leyi
def generate_lowest_ratio():
    pipeline = [
        {
            '$group': {
                '_id': '$agency_name',
                'total_budget_expense': {'$sum': {'$toDouble': '$adopted_budget_amount'}},
                'total_financial_planning': {'$sum': {'$toDouble': '$financial_plan_amount'}}
            }
        },
        {
            '$match': {
                'total_financial_planning': {'$ne': 0}  
            }
        },
        {
            '$project': {
                'agency_name': '$_id',
                'budget_expense': '$total_budget_expense',
                'financial_planning': '$total_financial_planning',
                'expense_to_planning_ratio': {'$divide': ['$total_budget_expense', '$total_financial_planning']}
            }
        },
        {
            '$sort': {
                'expense_to_planning_ratio': 1   
            }
        },
        {
            '$limit': 10
        }
    ]

    result = collection.aggregate(pipeline)
    agency_names = []
    expense_to_planning_ratios = []
    for doc in result:
        agency_names.append(doc['agency_name'])
        expense_to_planning_ratios.append(doc['expense_to_planning_ratio'])

    fig = go.Figure([go.Bar(x=agency_names, y=expense_to_planning_ratios)])

    fig.update_layout(
        title='Top 10 Agencies by Lowest Expense-to-Planning Ratio',
        xaxis_title='Agency Name',
        yaxis_title='Expense-to-Planning Ratio',
        hovermode='x', 
    )

    y_range_padding = 0.1
    fig.update_yaxes(range=[0, max(expense_to_planning_ratios, default=1) * (1 + y_range_padding)])

    return fig

### Flask Web Application Implementation

This Flask web application is designed to provide a user-friendly interface for analyzing and visualizing NYC government budget expenses. The app allows users to retrieve and filter records, as well as generate interactive visualizations for in-depth analysis. Below are the pages that are in the application:

- **SQL Schema Explanation**: The `/schema` route renders a page that explains the SQL schema used in the project, offering users insights into the database design and structure.

- **Record Retrieval**: The `/record-retrieving` route allows users to filter and retrieve budget expense records based on various criteria such as fiscal year, agency, and budget code. The filtered data is displayed in a user-friendly format, and the filter options are dynamically populated based on available data.

- **Plot Analysis**: The `/plot-analysis` route provides access to a range of visualizations that help users analyze budget trends and agency performance. Users can select a fiscal year or agency to generate specific visualizations, such as the top 10 agencies by budget, an agency's budget trend over time, and comparisons of expense-to-planning ratios.

This Flask app serves as a powerful tool for budget analysis, helping stakeholders make informed decisions based on comprehensive data insights.


In [22]:
from flask import Flask, render_template, request
from pymongo import MongoClient
import psycopg2
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import plot


app = Flask('NYC Expense Data',static_folder='static', template_folder="templates")

#mangoDB connection
client = MongoClient("localhost", 27017)
db = client.ManagingData_database
collection = db.expense_collection

#Postgresql Connection
conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="my_db",
    user="postgres",
    password="123")

#welcome page
@app.route('/')
def welcome_page():
    return render_template('welcome.html')

#SQL design explanations
@app.route('/schema')
def analysis():
    return render_template('sql_schema.html')

#record retrieving
@app.route('/record-retrieving', methods=['GET', 'POST'])
def record_retrieving():
    if request.method == 'POST':
        filters = {
            'fiscal_year': request.form.get('fiscal_year'),
            'agency': request.form.get('agency'),
            'unit_appropriation_name': request.form.get('unit_appropriation_name'),
            'budget_code_name': request.form.get('budget_code_name'),
            'object_class_name': request.form.get('object_class_name'),
            'object_code_name': request.form.get('object_code_name'),
            'responsibility_center_name': request.form.get('responsibility_center_name')
        }

        # Get the filtered records based on the selected criteria
        expense_records = get_filtered_records(filters)

        # Get filter options again for the dropdown filters
        fiscal_years, agencies, unit_appropriation_names, budget_code_names, object_class_names, object_code_names, responsibility_center_names = get_filter_options()

        return render_template('record_retrieving.html', records=expense_records, filters=filters,
                               fiscal_years=fiscal_years, agencies=agencies,
                               unit_appropriation_names=unit_appropriation_names, budget_code_names=budget_code_names,
                               object_class_names=object_class_names, object_code_names=object_code_names,
                               responsibility_center_names=responsibility_center_names)

    # Fetch filter options from the database to populate the dropdown filters
    fiscal_years, agencies, unit_appropriation_names, budget_code_names, object_class_names, object_code_names, responsibility_center_names = get_filter_options()

    return render_template('record_retrieving.html', fiscal_years=fiscal_years, agencies=agencies,
                           unit_appropriation_names=unit_appropriation_names, budget_code_names=budget_code_names,
                           object_class_names=object_class_names, object_code_names=object_code_names,
                           responsibility_center_names=responsibility_center_names)

#plot analysis
@app.route('/plot-analysis', methods=['GET', 'POST'])
def plot_analysis():
    if request.method == 'POST':
        fiscal_year = request.form.get('fiscal_year')
        agency_name = request.form.get('agency_name')

    else:
        fiscal_year = '2023'  # Set a default fiscal year as a string
        agency_name = 'DEPARTMENT OF EDUCATION'

    fiscal_year = fiscal_year or '2023'
    agency_name = agency_name or 'DEPARTMENT OF EDUCATION'
    # Generate the Plotly chart for top 10 agencies
    top_10_agencies_graph_html = top_agencies(fiscal_year)
    # Get unique fiscal years for the dropdown filter
    fiscal_years = sorted(collection.distinct('fiscal_year'))
    

    if agency_name:
        # Generate the Plotly chart for the agency's expense trend
        agency_graph_html = get_agency_budget_by_fiscal_year(agency_name) or '2023'
        # Get unique agency names for the dropdown filter
        agency_names = sorted(collection.distinct('agency_name'))
        return render_template('plot_analysis.html', 
                               top_10_agencies_graph_html=top_10_agencies_graph_html,
                               agency_graph_html=agency_graph_html,
                               fiscal_year=fiscal_year,
                               fiscal_years=fiscal_years,
                               agency_name=agency_name,
                               agency_names=agency_names)
    else:
        # Get agency names for the dropdown filter
        agency_names = sorted(collection.distinct('agency_name'))

        try:
            agency_graph_html
        except NameError:
            agency_graph_html = ""

        return render_template('plot_analysis.html', 
                               top_10_agencies_graph_html=top_10_agencies_graph_html,
                               fiscal_year=fiscal_year,
                               fiscal_years=fiscal_years,
                               agency_names=agency_names,
                               agency_graph_html=agency_graph_html)



@app.route('/plot-analysis/top-10-agencies')
def top_10_agencies():
    #retrieve graph
    fig_top_10_agencies = get_top_10_agencies()
    # Generate the HTML representation of the chart
    graph_html = plot(fig_top_10_agencies, output_type='div')
    return render_template('top_10_agency.html', graph_html=graph_html)

@app.route('/plot-analysis/Highest-Expense-to-Planning-Ratio')
def highest_ratio():
    fig_agency_without_growth = higest_ratio()
    graph_html = plot(fig_agency_without_growth, output_type='div')
    return render_template('highest_ratio.html', graph_html=graph_html)
    
@app.route('/plot-analysis/Lowest-Expense-to-Planning-Ratio')
def plot_lowest_ratio():
    fig_agency_with_growth= generate_lowest_ratio() 
    graph_html = plot(fig_agency_with_growth, output_type='div')
    return render_template('lowest_ratio.html', graph_html=graph_html)

@app.route('/plot-analysis/without-growth')
def agency_without_growth():
    return render_template('agency_without_growth.html')


if __name__ == '__main__':
    app.run(host="localhost", port=5001)



 * Serving Flask app 'NYC Expense Data'
 * Debug mode: off


 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [08/Aug/2023 05:52:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 05:53:16] "GET /schema HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 05:53:16] "GET /static/schema.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [08/Aug/2023 05:53:23] "GET /record-retrieving HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 05:53:43] "POST /record-retrieving HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 05:53:53] "POST /record-retrieving HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 05:53:59] "GET /plot-analysis HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 05:54:10] "POST /plot-analysis HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 05:54:25] "POST /plot-analysis HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 05:54:40] "GET /plot-analysis/top-10-agencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 05:55:12] "GET /plot-analysis HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2023 05:55:15] "GET /plot-analysis/Highest-Expense-to-Planning-Ratio HTTP/1.1" 200 -
127.0.0.1 - - [0